# Introduction to DataFrames
**[Bogumił Kamiński](http://bogumilkaminski.pl/about/), December 8, 2019**

In [1]:
using DataFrames
using BenchmarkTools

## Performance tips

### Access by column number is faster than by name

In [2]:
x = DataFrame(rand(5, 1000))
@btime $x[!, 500];
@btime $x.x500;

  4.399 ns (0 allocations: 0 bytes)
  13.727 ns (0 allocations: 0 bytes)


### When working with data `DataFrame` use barrier functions or type annotation

In [3]:
using Random
function f_bad() # this function will be slow
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y, z = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_bad();
# if you run @code_warntype f_bad() then you notice
# that Julia does not know column types of `DataFrame`


  84.487 ms (5999020 allocations: 122.06 MiB)


In [4]:
# solution 1 is to use barrier function (it should be possible to use it in almost any code)
function f_inner(y,z)
   p = 0.0
   for i in 1:length(y)
       p += y[i]*z[i]
   end
   p
end

function f_barrier() # extract the work to an inner function
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    f_inner(x[!, 1], x[!, 2])
end

using LinearAlgebra
function f_inbuilt() # or use inbuilt function if possible
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    dot(x[!, 1], x[!, 2])
end

@btime f_barrier();
@btime f_inbuilt();

  11.836 ms (42 allocations: 30.52 MiB)
  11.448 ms (42 allocations: 30.52 MiB)


In [5]:
# solution 2 is to provide the types of extracted columns
# it is simpler but there are cases in which you will not know these types
# This example  assumes that you have DataFrames master at least from August 31, 2018
function f_typed()
    Random.seed!(1); x = DataFrame(rand(1000000,2))
    y::Vector{Float64}, z::Vector{Float64} = x[!, 1], x[!, 2]
    p = 0.0
    for i in 1:nrow(x)
        p += y[i]*z[i]
    end
    p
end

@btime f_typed();

  11.756 ms (42 allocations: 30.52 MiB)


### Consider using delayed `DataFrame` creation technique

also notice the difference in performance between `DataFrame` and `DataFrame!` (copying vs non-copying data frame creation)

In [6]:
function f1()
    x = DataFrame!([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f1a()
    x = DataFrame([Vector{Float64}(undef, 10^4) for i in 1:100]) # we work with a DataFrame directly
    for c in 1:ncol(x)
        d = x[!, c]
        for r in 1:nrow(x)
            d[r] = rand()
        end
    end
    x
end

function f2()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame!(x) # we delay creation of DataFrame after we have our job done
end

function f2a()
    x = Vector{Any}(undef, 100)
    for c in 1:length(x)
        d = Vector{Float64}(undef, 10^4)
        for r in 1:length(d)
            d[r] = rand()
        end
        x[c] = d
    end
    DataFrame(x) # we delay creation of DataFrame after we have our job done
end

@btime f1();
@btime f1a();
@btime f2();
@btime f2a();

  23.668 ms (1949736 allocations: 37.42 MiB)
  25.578 ms (1949936 allocations: 45.05 MiB)
  5.804 ms (836 allocations: 7.68 MiB)
  6.458 ms (1036 allocations: 15.32 MiB)


### You can add rows to a `DataFrame` in place and it is fast

In [7]:
x = DataFrame(rand(10^6, 5))
y = DataFrame(transpose(1.0:5.0))
z = [1.0:5.0;]

@btime vcat($x, $y); # creates a new DataFrame - slow
@btime append!($x, $y); # in place - fast

x = DataFrame(rand(10^6, 5)) # reset to the same starting point
@btime push!($x, $z); # add a single row in place - fast

  12.832 ms (172 allocations: 38.16 MiB)
  834.233 ns (18 allocations: 768 bytes)
  316.517 ns (16 allocations: 256 bytes)


### Allowing `missing` as well as `categorical` slows down computations

In [8]:
using StatsBase

function test(data) # uses countmap function to test performance
    println(eltype(data))
    x = rand(data, 10^6)
    y = categorical(x)
    println(" raw:")
    @btime countmap($x)
    println(" categorical:")
    @btime countmap($y)
    nothing
end

test(1:10)
test([randstring() for i in 1:10])
test(allowmissing(1:10))
test(allowmissing([randstring() for i in 1:10]))


Int64
 raw:
  4.712 ms (7 allocations: 7.63 MiB)
 categorical:
  17.635 ms (4 allocations: 608 bytes)
String
 raw:
  22.013 ms (4 allocations: 608 bytes)
 categorical:
  29.232 ms (4 allocations: 608 bytes)
Union{Missing, Int64}
 raw:
  10.345 ms (4 allocations: 624 bytes)
 categorical:
  17.681 ms (4 allocations: 608 bytes)
Union{Missing, String}
 raw:
  32.312 ms (4 allocations: 608 bytes)
 categorical:
  40.473 ms (4 allocations: 608 bytes)


### When aggregating use column selector and prefer categorical or pooled array grouping variable

In [9]:
df = DataFrame(x=rand('a':'d', 10^7), y=1);

In [10]:
@btime by($df, :x, v -> sum(v.y)) # traditional syntax, slow

  247.536 ms (232 allocations: 433.19 MiB)


,x,x1
,Char,Int64
1,'b',2500951
2,'a',2497716
3,'c',2500955
4,'d',2500378


In [11]:
@btime by($df, :x, :y=>sum) # use column selector

  194.269 ms (178 allocations: 356.89 MiB)


,x,y_sum
,Char,Int64
1,'b',2500951
2,'a',2497716
3,'c',2500955
4,'d',2500378


In [12]:
categorical!(df, :x);

In [13]:
@btime by($df, :x, :y=>sum)

  95.231 ms (195 allocations: 152.60 MiB)


,x,y_sum
,Categorical…,Int64
1,'a',2497716
2,'b',2500951
3,'c',2500955
4,'d',2500378


In [14]:
using PooledArrays

In [15]:
df.x = PooledArray{Char}(df.x)

10000000-element PooledArray{Char,UInt8,1,Array{UInt8,1}}:
 'b'
 'a'
 'a'
 'b'
 'c'
 'c'
 'b'
 'b'
 'a'
 'b'
 'c'
 'd'
 'a'
 ⋮  
 'a'
 'c'
 'd'
 'a'
 'b'
 'a'
 'c'
 'c'
 'a'
 'c'
 'c'
 'd'

In [16]:
@btime by($df, :x, :y=>sum)

  95.332 ms (188 allocations: 152.60 MiB)


,x,y_sum
,Char,Int64
1,'b',2500951
2,'a',2497716
3,'c',2500955
4,'d',2500378


### Use views instead of materializing a new DataFrame

In [17]:
x = DataFrame(rand(100, 1000))

,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962
2,0.814225,0.798912,0.261771,0.911435,0.00748892,0.356792,0.0631944,0.60062
3,0.765845,0.650583,0.493127,0.0861466,0.332161,0.0910342,0.554393,0.53809
4,0.0466681,0.0592616,0.344917,0.394735,0.320023,0.90514,0.661371,0.957423
5,0.451083,0.767918,0.079251,0.38842,0.965302,0.486402,0.605539,0.778464
6,0.238854,0.670357,0.704591,0.735974,0.850833,0.178727,0.603458,0.212492
7,0.0540254,0.543548,0.327693,0.675549,0.16487,0.377317,0.302322,0.898442
8,0.16169,0.358802,0.890735,0.499997,0.52147,0.556091,0.62327,0.683097
9,0.386167,0.976076,0.0990527,0.944734,0.973824,0.587316,0.499486,0.260115


In [18]:
@btime $x[1:1, :]

  99.099 μs (1510 allocations: 194.39 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962


In [19]:
@btime $x[1, :]

  26.130 ns (1 allocation: 32 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962


In [20]:
@btime view($x, 1:1, :)

  26.433 ns (1 allocation: 48 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962


In [21]:
@btime $x[1:1, 1:20]

  4.143 μs (54 allocations: 7.22 KiB)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962


In [22]:
@btime $x[1, 1:20]

  35.685 ns (2 allocations: 80 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962


In [23]:
@btime view($x, 1:1, 1:20)

  36.391 ns (2 allocations: 96 bytes)


,x1,x2,x3,x4,x5,x6,x7,x8
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.401231,0.554653,0.351137,0.466443,0.146239,0.491392,0.738363,0.381962
